In [ ]:
import pandas as pd
import random

#for easy reading
import textwrap
t_width = 120

#load in the blank df with all the models' summaries
all_summaries = pd.read_csv('all_summaries_with_abstract.csv')
summary_names = all_summaries.columns.values.tolist()
metric_names = ['grammar', 'is_summarizing', 'lay_readability', 'relative_rank']
for col_name in summary_names:
  position = all_summaries.columns.get_loc(col_name) + 1
  for metric_name in metric_names[::-1]:
    all_summaries.insert(position, f'{col_name}-{metric_name}', '')
all_summaries

In [ ]:
#given a df and an index of a paper, prompt the user for evaluation of the models
def human_eval(df, row_index, summary_names, scale_metric_names):
  #randomizing and hiding the names of the models to prevent bias
  rand_sum_names = random.sample(summary_names, len(summary_names))

  for i in range(len(rand_sum_names)):
    summary_name = rand_sum_names[i]
    wrapped_text = textwrap.fill(f'Summary {i+1}: {df[summary_name][row_index]}', width=t_width)
    print(wrapped_text)
    #prompting the evaluator for each scalar metric
    for scale_metric_name in scale_metric_names:
      score = int(input(f'enter score for {scale_metric_name} (int 1-5)\n'))
      df.at[row_index, f'{summary_name}-{scale_metric_name}'] = score
    print('\n\n')

  #printing out the papers again for easy comparision in the ranking evaluation
  for i in range(len(rand_sum_names)):
    summary_name = rand_sum_names[i]
    wrapped_text = textwrap.fill(f'Summary {i+1}: {df[summary_name][row_index]}\n\n\n', width=t_width)
    print(wrapped_text)

  #prompting the evaluator for ranking
  for i in range(len(rand_sum_names)):
    summary_name = rand_sum_names[i]
    ranking = int(input(f'enter ranking for summary {i+1} (int 1-{len(rand_sum_names)})\n'))
    df.at[row_index, f'{summary_name}-relative_rank'] = ranking
  display(df.iloc[row_index])

In [ ]:
#entered for each row
human_eval(all_summaries, 'put your index here', summary_names, ['grammar', 'is_summarizing', 'lay_readability'])

In [ ]:
#pickling and saving the evaluation result
all_summaries.to_pickle("./checkpoint.pkl")

In [ ]:
#loading in each of our datasets
jordy = all_summaries.loc[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]
jordy

In [ ]:
andy = pd.read_pickle("./checkpoint25-33.pkl")
andy = andy.loc[[25,26,27,28,29,30,31,32,33]]
andy

In [ ]:
ahmed = pd.read_pickle("./checkpoint50-59_and_9.pkl")
ahmed = ahmed.loc[[50,51,52,53,54,55,56,57,58,59]]
for summary_name in summary_names:
  ahmed = ahmed.rename(columns={f'{summary_name}-sensical': f'{summary_name}-grammar'})
  ahmed[f'{summary_name}-relative_rank'] = ahmed[f'{summary_name}-relative_rank'].map({1: 10, 2: 9, 3: 8, 4: 7, 5: 6, 6: 5, 7: 4, 8: 3, 9: 2, 10: 1})
ahmed

In [ ]:
jerry = pd.read_pickle("./checkpoint-75-85.pkl")
jerry = jerry.loc[[75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85]]
jerry

In [ ]:
#combining our datasets
all_our_summaries = pd.concat([jordy, andy, jerry, ahmed])
for summary_name in summary_names:
  all_our_summaries = all_our_summaries.rename(columns={f'{summary_name}-grammar': f'{summary_name}-sensical'})
all_our_summaries.to_pickle("./all_summaries.pkl")

In [ ]:
all_our_summaries

In [ ]:
all_our_summaries[f'ground_truth-grammar'].mean()

4.05

In [ ]:
#calculating averages
average_scores = pd.DataFrame(columns=summary_names)
new_metric_names = ['sensical', 'is_summarizing', 'lay_readability', 'relative_rank']
for metric_name in new_metric_names:
  for summary_name in summary_names:
    column_name = f"{summary_name}-{metric_name}"
    technique_scores = all_our_summaries[column_name]
    median_score = technique_scores.median()
    mean_score = technique_scores.mean()
    average_scores.loc[f"Median {metric_name}", summary_name] = median_score
    average_scores.loc[f"Mean {metric_name}", summary_name] = mean_score
average_scores

,ground_truth,abstract,leadk_syn_base,gpt_base,non_ft_xLarge,non_ft_xBase,xBase_2k,xBase_4k,xLarge_2k,xLarge_4k
Median sensical,4.0,4.0,2.5,4.0,3.0,1.0,3.0,3.0,3.0,3.0
Mean sensical,4.05,4.075,2.8,4.325,2.925,1.0,3.0,3.1,3.05,3.25
Median is_summarizing,3.5,4.0,2.0,4.0,2.0,1.0,3.0,3.0,3.0,3.0
Mean is_summarizing,3.7,3.875,2.3,3.85,1.975,1.0,3.175,3.1,3.025,3.475
Median lay_readability,3.0,2.0,2.0,3.0,2.0,1.0,4.0,4.0,4.0,3.5
Mean lay_readability,3.425,2.35,1.875,2.875,2.1,1.0,3.575,3.5,3.675,3.475
Median relative_rank,8.0,8.0,3.0,8.0,3.0,1.0,5.0,6.0,6.0,7.0
Mean relative_rank,7.525,7.525,3.575,7.775,3.325,1.0,5.5,5.825,5.9,6.6


In [ ]:
import os
import json
#save to json
json_file_path = os.path.join('/content/', 'all_summaries_json')
all_our_summaries.to_json(json_file_path, orient="records", lines=True)
json_file_path = os.path.join('/content/', 'average_scores_json')
average_scores.to_json(json_file_path, orient="records", lines=True)
all_our_summaries.to_csv('all_summaries_csv.csv')

In [ ]:
#reading from json
summariesFromJSON = pd.read_json('all_summaries', lines = True)
summariesFromJSON

ValueError: ignored